In [ ]:
!pip install -r requirements.txt
!pip install opencv-python
!pip install mediapipe

## Issue(s)

- Data Feature Extraction -> Şafak
- Encoder, Decoder, VQ-VAE
  - [ ] Codebook (Different VQ-VAE) FFN -> Şilan
    - [ ] Added quantization saving at notebook (under analyze/quantization)
  - [ ] Encoder-Decoder (FFN, CNN, GNN) -> Metehan
- [ ] Transformer Encoder (Pre-training) -> Karahan

In [1]:
import os
import cv2 
import sys
import glob
import time
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from vector_quantize_pytorch import VectorQuantize, ResidualVQ

from lib.config import *
from lib.encoder.vqvae import VQVAE
from lib.utils.pose import get_pose_estimation
from lib.encoder.ffn import FFNEncoder, FFNDecoder
from lib.encoder.cnn import CNNEncoder, CNNDecoder
from lib.data.dataset import PoseDataset, PoseDistanceDataset
from lib.train.run_autoencoder_training import AutoencoderTrainer

### 1. Pose Inference

In [ ]:
SAMPLE = 'dataset/corpus/ABARTMAK_0.mp4'
SAMPLE_POSE = get_pose_estimation(SAMPLE)

In [13]:
import pandas as pd

def get_pose_array(SAMPLE_POSE):
    """Converts the pose data into a numpy array
    """

    POSE_RAW = pd.DataFrame(SAMPLE_POSE['pose'])
    RIGHT_HAND_RAW = pd.DataFrame(SAMPLE_POSE['right'])
    LEFT_HAND_RAW = pd.DataFrame(SAMPLE_POSE['left'])

    POSE_DF = {}

    for col in POSE_RAW.columns:
        POSE_DF[ 'POSE_' + col + '_X'] = POSE_RAW[col].apply(lambda x: x[0])
        POSE_DF[ 'POSE_' + col + '_Y'] = POSE_RAW[col].apply(lambda x: x[1])
        POSE_DF[ 'POSE_' + col + '_Z'] = POSE_RAW[col].apply(lambda x: x[2])
        # POSE_DF[col + '_viz'] = POSE_RAW[col].apply(lambda x: x[3])

    for col in RIGHT_HAND_RAW.columns:
        POSE_DF[ 'RIGHT_' + col + '_X' ] = RIGHT_HAND_RAW[col].apply(lambda x: x[0])
        POSE_DF[ 'RIGHT_' + col + '_Y' ] = RIGHT_HAND_RAW[col].apply(lambda x: x[1])
        POSE_DF[ 'RIGHT_' + col + '_Z' ] = RIGHT_HAND_RAW[col].apply(lambda x: x[2])
        # POSE_DF['RIGHT_' + col + '_viz'] = RIGHT_HAND_RAW[col].apply(lambda x: x[3])

    for col in LEFT_HAND_RAW.columns:
        POSE_DF[ 'LEFT_' + col + '_X' ] = LEFT_HAND_RAW[col].apply(lambda x: x[0])
        POSE_DF[ 'LEFT_' + col + '_Y' ] = LEFT_HAND_RAW[col].apply(lambda x: x[1])
        POSE_DF[ 'LEFT_' + col + '_Z' ] = LEFT_HAND_RAW[col].apply(lambda x: x[2])
        # POSE_DF['LEFT_' + col + '_viz'] = LEFT_HAND_RAW[col].apply(lambda x: x[3])

    POSE_DF = pd.DataFrame(POSE_DF)

    return POSE_DF

In [14]:
POSE_DF = get_pose_array(SAMPLE_POSE[0])

In [15]:
POSE_DF = POSE_DF.replace(np.nan,0)

In [16]:
def get_matrices(POSE_DF):
    """Converts the pose data into a numpy array of distance matrices
    """
    x_cols = [col for col in POSE_DF.columns if col.endswith('_X')]
    y_cols = [col for col in POSE_DF.columns if col.endswith('_Y')]
    z_cols = [col for col in POSE_DF.columns if col.endswith('_Z')]

    frames = []
    for i in range(1, POSE_DF.shape[0]):
        x_row = POSE_DF[x_cols].iloc[i].to_numpy()
        y_row = POSE_DF[y_cols].iloc[i].to_numpy()
        z_row = POSE_DF[z_cols].iloc[i].to_numpy()

        def get_difference_matrix(row):
            m, n = np.meshgrid(row, row)
            out = m-n
            return out

        x_diff = get_difference_matrix(x_row)
        y_diff = get_difference_matrix(y_row)
        z_diff = get_difference_matrix(z_row)

        frame = np.stack([x_diff, y_diff, z_diff], axis=2)
        frames.append(frame)

    frames = np.stack(frames, axis=0)
    return frames

In [17]:
OUT_PATH = 'dataset/adjacency'
POSE_PATH = 'dataset/pose'

for file in tqdm(glob.glob('dataset/pose/*.npy')):
    if os.path.exists(os.path.join(OUT_PATH, os.path.basename(file).replace('.mp4', '.npy'))):
        # print('Skipping', file)
        continue
    with open(file, 'rb') as f:
        array = np.load(f, allow_pickle=True)
        # replace nan with 0 
        array = np.nan_to_num(array)
    pose_df = pd.DataFrame(array, columns=POSE_DF.columns)
    pose_df = pose_df.replace(np.nan,0)
    MATRICES = get_matrices(pose_df)
    # print(MATRICES.shape)
    np.save(os.path.join(OUT_PATH, os.path.basename(file).replace('.mp4', '.npy')), MATRICES)


0it [00:00, ?it/s]

In [37]:
print(pd.DataFrame(x_diff, columns=[col for col in POSE_DF.columns if col.endswith('_X')], index=[col for col in POSE_DF.columns if col.endswith('_X')]).to_markdown())

|                           |   POSE_NOSE_X |   POSE_LEFT_EYE_INNER_X |   POSE_LEFT_EYE_X |   POSE_LEFT_EYE_OUTER_X |   POSE_RIGHT_EYE_INNER_X |   POSE_RIGHT_EYE_X |   POSE_RIGHT_EYE_OUTER_X |   POSE_LEFT_EAR_X |   POSE_RIGHT_EAR_X |   POSE_MOUTH_LEFT_X |   POSE_MOUTH_RIGHT_X |   POSE_LEFT_SHOULDER_X |   POSE_RIGHT_SHOULDER_X |   POSE_LEFT_ELBOW_X |   POSE_RIGHT_ELBOW_X |   POSE_LEFT_WRIST_X |   POSE_RIGHT_WRIST_X |   POSE_LEFT_PINKY_X |   POSE_RIGHT_PINKY_X |   POSE_LEFT_INDEX_X |   POSE_RIGHT_INDEX_X |   POSE_LEFT_THUMB_X |   POSE_RIGHT_THUMB_X |   POSE_LEFT_HIP_X |   POSE_RIGHT_HIP_X |   POSE_LEFT_KNEE_X |   POSE_RIGHT_KNEE_X |   POSE_LEFT_ANKLE_X |   POSE_RIGHT_ANKLE_X |   POSE_LEFT_HEEL_X |   POSE_RIGHT_HEEL_X |   POSE_LEFT_FOOT_INDEX_X |   POSE_RIGHT_FOOT_INDEX_X |   RIGHT_WRIST_X |   RIGHT_THUMB_CMC_X |   RIGHT_THUMB_MCP_X |   RIGHT_THUMB_IP_X |   RIGHT_THUMB_TIP_X |   RIGHT_INDEX_FINGER_MCP_X |   RIGHT_INDEX_FINGER_PIP_X |   RIGHT_INDEX_FINGER_DIP_X |   RIGHT_INDEX_FINGER_TIP_X

In [ ]:
import glob
import numpy as np
from tqdm.notebook import tqdm
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    ARRAY_PATH = 'dataset/pose/'
    for datapath in tqdm(glob.glob('dataset/corpus/*.mp4')):
        print(datapath)
        pose, _ = get_pose_estimation(datapath)
        pose_array = get_pose_array(pose)
        print(pose_array.shape, datapath)
        dname = datapath.split('/')[-1].replace('.mp4', '.npy')
        with open(ARRAY_PATH+'/'+dname, 'wb') as f:
            np.save(f, pose_array)

### 2. Graph Autoencoder Training

In [2]:
DATA_PATH = 'dataset/pose/'
data = glob.glob(DATA_PATH + '*.npy')
X_train, X_val = train_test_split(data, test_size=0.2, random_state=42)

In [3]:
train_dataset = PoseDataset(X_train)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=GLOBAL_CONFIG.BATCH_SIZE, 
    shuffle=True,
)

In [4]:
val_dataset = PoseDataset(X_val)
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=GLOBAL_CONFIG.BATCH_SIZE, 
    shuffle=True,
)

In [5]:
torch.cuda.is_available()

False

In [6]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_ENCODER = FFNEncoder(
    input_dim=GLOBAL_CONFIG.MODEL_ENCODER_INPUT_DIM,
    hidden_dim=GLOBAL_CONFIG.MODEL_ENCODER_HIDDEN_DIM,
    output_dim=GLOBAL_CONFIG.MODEL_ENCODER_OUTPUT_DIM,
)

MODEL_DECODER = FFNDecoder(
    input_dim=GLOBAL_CONFIG.MODEL_DECODER_INPUT_DIM,
    hidden_dim=GLOBAL_CONFIG.MODEL_DECODER_HIDDEN_DIM,
    output_dim=GLOBAL_CONFIG.MODEL_ENCODER_INPUT_DIM,
)

MODEL_QUANT = ResidualVQ(
    dim = GLOBAL_CONFIG.MODEL_VQ_EMBED_DIM,
    stochastic_sample_codes=True,
    num_quantizers=1,      # specify number of quantizers
    codebook_size=GLOBAL_CONFIG.MODEL_VQ_NUM_EMBS,    # codebook size           
    kmeans_init=True,   # set to True
    kmeans_iters=100     # number of kmeans iterations to calculate the centroids for the codebook on init
)

MODEL_VQVAE = VQVAE(
    encoder=MODEL_ENCODER,
    decoder=MODEL_DECODER,
    vq=MODEL_QUANT,
)

trainer = AutoencoderTrainer(
    model=MODEL_VQVAE,
    learning_rate=GLOBAL_CONFIG.LEARNING_RATE,
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader,
    num_epochs=GLOBAL_CONFIG.NUM_EPOCHS,
    device='cpu',
)

In [ ]:
trainer.train()

In [ ]:
from tqdm.notebook import tqdm

MODEL_VQVAE.eval()

dfs = []
for train_sample in tqdm(train_dataloader):
    with torch.no_grad():
        quantized, indices, commitment_loss = MODEL_VQVAE(train_sample['array'].float())
        dfs.append(pd.DataFrame({
            'videos': train_sample['token'],
            'labels': indices.detach().cpu().numpy().reshape(-1),
            'frame': train_sample['frame'].detach().cpu().numpy().reshape(-1)
        }))

In [ ]:
df = pd.concat(dfs)

[412, 385, 805]

In [10]:
df

,videos,labels,frame
0,TICARET_0.npy,663,8
1,EL_0.npy,663,1
2,KAN_0.npy,375,31
3,EL O╠êPMEK_1.npy,663,26
4,SALDIRMAK_0.npy,282,15
...,...,...,...
0,S╠ğOK _0.npy,96,40
1,SOHBET ETMEK_0.npy,882,6
2,YAS╠ğAMAK_0.npy,96,47
3,GU╠êC╠ğLU╠ê _1.npy,282,7


In [11]:
df.labels.value_counts()

labels
629    46223
96     43641
882    23308
663    21702
282    11906
375     4364
0        141
Name: count, dtype: int64

In [14]:
import cv2
for rec in tqdm(df[df['labels'] == 375].to_dict(orient='records')[:100]):
    # save frame video to disk
    video = rec['videos'].split('.')[0]
    video_path = f"dataset/corpus/{video}.mp4"
    frame_idx = rec['frame']
    label = rec['labels']
    
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    
    if not os.path.exists(f'analyze/quantization/{label}'): os.mkdir(f'analyze/quantization/{label}')

    for i in range(frame_idx):
        ret, frame = cap.read()
        if i == frame_idx-1:
            cv2.imwrite(f'analyze/quantization/{label}/{video}_{frame_idx}.jpg', frame)   

  0%|          | 0/100 [00:00<?, ?it/s]

###  3. 3D-CNN Training 

In [15]:
DATA_PATH = 'dataset/adjacency/'
data = glob.glob(DATA_PATH + '*.npy')[:100]
X_train, X_val = train_test_split(data, test_size=0.2, random_state=42)

In [16]:
train_dataset = PoseDistanceDataset(X_train)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=GLOBAL_CONFIG.BATCH_SIZE, 
    shuffle=True,
    collate_fn=PoseDistanceDataset.collate_fn
)

100%|██████████| 80/80 [00:00<00:00, 249.82it/s]


In [17]:
val_dataset = PoseDistanceDataset(X_val)
val_dataloader = DataLoader(
    val_dataset, 
    batch_size=GLOBAL_CONFIG.BATCH_SIZE, 
    shuffle=True,
    collate_fn=PoseDistanceDataset.collate_fn
)

100%|██████████| 20/20 [00:00<00:00, 187.17it/s]


In [18]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

MODEL_ENCODER = CNNEncoder(
    input_channels=3,
)

MODEL_DECODER = CNNDecoder(
    output_channels=3,
)

MODEL_QUANT = ResidualVQ(
    dim = GLOBAL_CONFIG.MODEL_VQ_EMBED_DIM,
    stochastic_sample_codes=True,
    num_quantizers=1,      # specify number of quantizers
    codebook_size=GLOBAL_CONFIG.MODEL_VQ_NUM_EMBS,    # codebook size           
    kmeans_init=True,   # set to True
    kmeans_iters=10     # number of kmeans iterations to calculate the centroids for the codebook on init
)

MODEL_VQVAE = VQVAE(
    encoder=MODEL_ENCODER,
    decoder=MODEL_DECODER,
    vq=MODEL_QUANT,
)

trainer = AutoencoderTrainer(
    model=MODEL_VQVAE,
    learning_rate=GLOBAL_CONFIG.LEARNING_RATE,
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader,
    num_epochs=GLOBAL_CONFIG.NUM_EPOCHS,
    device='cpu',
)

In [20]:
trainer.train()

  1%|          | 1/100 [03:39<6:01:25, 219.05s/it]


    ***
    Epoch:1, 
    Train Commitment Loss: 0.014852691213778389, 
    Train Reconstruction Loss: 132.74135905504227, 
    Train Quantization Tokens:
    |       |   228 |   10 |   923 |   237 |   419 |   189 |   770 |   882 |   816 |   831 |   730 |   94 |   591 |   315 |   149 |   286 |   120 |   40 |   620 |   231 |   326 |   687 |   782 |   618 |   643 |   166 |   728 |   785 |   42 |   49 |   580 |   510 |   240 |   92 |   66 |   682 |   538 |   847 |   113 |   575 |   700 |   400 |   32 |   576 |   784 |   758 |   78 |   863 |   913 |   672 |   880 |   852 |   557 |   499 |   956 |   853 |   918 |   698 |   439 |   555 |   426 |   933 |   521 |   563 |   625 |   367 |   462 |   346 |   331 |   685 |   236 |   74 |   894 |   583 |   509 |   726 |   81 |   420 |   756 |   405 |   433 |   141 |   188 |   844 |   242 |   370 |   229 |   958 |   267 |   792 |   223 |   337 |   212 |   714 |   747 |   144 |   523 |   662 |   200 |   182 |   118 |   944 |   926 |   616 |   693 |  

  1%|          | 1/100 [05:38<9:19:14, 338.94s/it]


KeyboardInterrupt: 

In [21]:
from tqdm.notebook import tqdm

MODEL_VQVAE.eval()

dfs = []
for train_sample in tqdm(train_dataloader):
    with torch.no_grad():
        quantized, indices, commitment_loss = MODEL_VQVAE(train_sample['array'].float())
        dfs.append(pd.DataFrame({
            'videos': train_sample['tokens'],
            'labels': indices.detach().cpu().numpy().reshape(-1),
            'start_idx': train_sample['start_idx'],
            'end_idx': train_sample['end_idx']
        }))

  0%|          | 0/435 [00:00<?, ?it/s]

In [22]:
df = pd.concat(dfs)

In [23]:
df

,videos,labels,start_idx,end_idx
0,ANLAM_0,0,29,34
1,BENZIN_0,0,7,12
2,HAMAM_0,0,18,23
3,HAMAM_0,0,16,21
4,TUS╠ğLAMAK _2,0,60,65
...,...,...,...,...
2,FREN_0,0,28,33
3,DAVET _0,0,51,56
4,BOS╠ğ OTURMAK_0,0,29,34
5,ORAYA_0,0,35,40


In [24]:
df.labels.value_counts()

labels
0    3479
Name: count, dtype: int64

In [23]:
for rec in tqdm(df[df['labels'] == 233].to_dict(orient='records')):
    # save frame video to disk
    video = rec['videos'].split('.')[0]
    video_path = f"dataset/corpus/{video}.mp4"
    start_idx = rec['start_idx']
    end_idx = rec['end_idx']
    label = rec['labels']

    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    
    import os
    if not os.path.exists(f'analyze/quantization/{label}'):
        os.mkdir(f'analyze/quantization/{label}')

    #
    FRAMES = []
    for i in range(frame_idx):
        ret, frame = cap.read()
        if i >= start_idx and i <= end_idx:
            FRAMES.append(frame)

    # write frames to video
    out = cv2.VideoWriter(f'analyze/quantization/{label}/{video}_{start_idx}_{end_idx}.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, (frame.shape[1], frame.shape[0]))
    for frame in FRAMES:
        out.write(frame)

    out.release()      

0it [00:00, ?it/s]